In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from ta.momentum import RSIIndicator, StochasticOscillator
from ta.trend import MACD
from tqdm import tqdm

# Ticker symbols for the 12 stocks
tickers = [
    "JPM", "BAC", "WFC", "GS", "MS", "PNC",
    "BMO", "STT", "HSBC", "AXP", "FITB", "MTB"
]

# Download historical OHLCV from 2014-01-01 to 2023-12-31
def download_data(ticker):
    data = yf.download(ticker, start="2014-01-01", end="2023-12-31")
    data = data.dropna()
    return data

stock_data = {ticker: download_data(ticker) for ticker in tqdm(tickers)}

  0%|          | 0/12 [00:00<?, ?it/s]

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
100%|██████████| 12/12 [00:03<00:00,  3.08it/s]


In [2]:
def compute_features(df):
    df = df.copy()

    # Ensure Close, High, Low are Series (1D)
    close = df["Close"].squeeze()
    high = df["High"].squeeze()
    low = df["Low"].squeeze()
    volume = df["Volume"].squeeze()

    print(f"Close dtype: {type(close)}, shape: {close.shape}")

    # Daily return
    df["Return"] = close.pct_change()

    # 30-day rolling volatility (target)
    df["Volatility"] = df["Return"].rolling(window=30).std()

    # RSI (14 days)
    df["RSI"] = RSIIndicator(close=close, window=14).rsi()

    # Momentum (5 days)
    df["MOM"] = close - close.shift(5)

    # OBV
    df["OBV"] = (np.sign(close.diff()) * volume).fillna(0).cumsum()

    # MACD
    macd = MACD(close=close, window_slow=26, window_fast=12, window_sign=9)
    df["MACD_LINE"] = macd.macd()
    df["MACD_SIGNAL"] = macd.macd_signal()
    df["MACD_HIST"] = macd.macd_diff()

    # Stochastic Oscillator
    stoch = StochasticOscillator(high=high, low=low, close=close, window=14, smooth_window=3)
    df["STO_K"] = stoch.stoch()           # formerly %K
    df["STO_D"] = stoch.stoch_signal()    # formerly %D

    # Lagged volatilities (t-1 to t-6)
    for i in range(1, 7):
        df[f"Vol_t_{i}"] = df["Volatility"].shift(i)

    # Volatility t+1 (our target)
    df["Vol_target"] = df["Volatility"].shift(-1)

    # Drop rows with NaNs
    df = df.dropna()

    return df

In [3]:
import os
import pickle

feature_data = {}
for ticker in tqdm(tickers):
    feature_data[ticker] = compute_features(stock_data[ticker])

feature_data_path = "feature_data.pkl"

if os.path.exists(feature_data_path):
    print("📦 Loading saved feature data from feature_data.pkl...")
    with open(feature_data_path, "rb") as f:
        feature_data = pickle.load(f)
else:
    print("⚙️ Computing feature data...")
    feature_data = {ticker: compute_features(stock_data[ticker]) for ticker in tqdm(tickers)}
    with open(feature_data_path, "wb") as f:
        pickle.dump(feature_data, f)
    print("💾 Saved feature data to feature_data.pkl")

100%|██████████| 12/12 [00:00<00:00, 81.55it/s]

Close dtype: <class 'pandas.core.series.Series'>, shape: (2516,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2516,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2516,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2516,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2516,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2516,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2516,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2516,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2516,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2516,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2516,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2516,)
📦 Loading saved feature data from feature_data.pkl...


In [4]:
from arch import arch_model
import warnings

def add_garch_predictions(df, ticker=None, verbose=True):
    df = df.copy()
    returns = df["Return"].dropna().values
    preds = []
    window_size = 500
    scale_factor = 100  # recommended by arch package

    if verbose:
        print(f"\n🔍 GARCH modeling for {ticker} — total points: {len(returns)}")

    for i in range(window_size, len(returns)):
        train_window = returns[i-window_size:i] * scale_factor  # rescale

        try:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                model = arch_model(train_window, vol='Garch', p=1, q=1, dist='normal', rescale=False)
                model_fit = model.fit(disp="off")
                forecast = model_fit.forecast(horizon=1)
                pred_vol_scaled = np.sqrt(forecast.variance.values[-1][0])
                pred_vol = pred_vol_scaled / scale_factor  # unscale
        except Exception as e:
            if verbose:
                print(f"⚠️ Failed at i={i} — {e}")
            pred_vol = np.nan

        preds.append(pred_vol)

        if verbose and i % 250 == 0:
            print(f"  → Index {i} | Pred Vol (unscaled): {pred_vol:.5f}")

    full_preds = [np.nan] * window_size + preds
    df["GARCH_pred"] = full_preds

    before = len(df)
    df = df.dropna()
    after = len(df)

    if verbose:
        print(f"✅ Done {ticker} | Rows dropped: {before - after} | Final: {after} rows")

    return df

In [5]:
# === Try loading precomputed garch_data from disk ===
garch_data_path = "garch_data.pkl"

if os.path.exists(garch_data_path):
    print("📦 Loading saved GARCH data from garch_data.pkl...")
    with open(garch_data_path, "rb") as f:
        garch_data = pickle.load(f)
    print("✅ Loaded GARCH data successfully!")
else:
    print("⚙️ Computing GARCH data from scratch...")
    garch_data = {}
    for ticker in tickers:
        print(f"\n====================== {ticker} ======================")
        garch_data[ticker] = add_garch_predictions(feature_data[ticker], ticker=ticker)

    # Save to disk
    with open(garch_data_path, "wb") as f:
        pickle.dump(garch_data, f)
    print("💾 Saved GARCH data to garch_data.pkl")

📦 Loading saved GARCH data from garch_data.pkl...
✅ Loaded GARCH data successfully!


In [6]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
import numpy as np
import warnings
warnings.filterwarnings("ignore")

def evaluate(y_true, y_pred):
    return {
        "R2": r2_score(y_true, y_pred),
        "RMSE": mean_squared_error(y_true, y_pred, squared=False),
        "MSE": mean_squared_error(y_true, y_pred),
        "MAE": mean_absolute_error(y_true, y_pred),
    }

def train_ml_models_baseline(df, ticker="TICKER"):
    print(f"\n📈 Training ML models for {ticker}...")

    # Feature and target selection
    features = [
        'RSI', 'MOM', 'OBV', 'MACD_LINE', 'MACD_SIGNAL', 'MACD_HIST',
        'STO_K', 'STO_D',
        'Vol_t_1', 'Vol_t_2', 'Vol_t_3', 'Vol_t_4', 'Vol_t_5', 'Vol_t_6'
    ]

    X = df[features].copy()
    # Sanitize column names just in case LightGBM is sensitive
    X.columns = [str(col).replace("-", "_").replace("%", "PCT").replace(".", "_DOT_") for col in X.columns]

    y = df["Vol_target"]

    # Static train-test split (same as paper: 2014–2020 train, 2021–2023 test)
    split_idx = int(len(df) * 0.7)
    X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
    y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

    models = {
        "KNN": KNeighborsRegressor(),
        "AdaBoost": AdaBoostRegressor(),
        "CatBoost": CatBoostRegressor(verbose=0),
        #"LightGBM": LGBMRegressor(),
        "XGBoost": XGBRegressor(verbosity=0),
        "RandomForest": RandomForestRegressor()
    }

    results = {}

    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        metrics = evaluate(y_test, y_pred)
        results[name] = metrics
        print(f"✅ {name} — R²: {metrics['R2']:.4f}, RMSE: {metrics['RMSE']:.4f}, MAE: {metrics['MAE']:.4f}")

    return results

/shared/home/ads066/.local/lib/python3.11/site-packages/xgboost/core.py:377: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc >= 2.28) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(


In [7]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def evaluate(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    return {
        "R2": r2_score(y_true, y_pred),
        "RMSE": np.sqrt(mse),
        "MSE": mse,
        "MAE": mean_absolute_error(y_true, y_pred),
    }

def train_all_stocks_ml_baseline(garch_data_dict, results_path="ml_baseline_results.csv"):
    # Check if results already exist
    if os.path.exists(results_path):
        print(f"📦 Loading existing results from {results_path}...")
        return pd.read_csv(results_path)

    final_results = []

    for ticker, df in garch_data_dict.items():
        print(f"\n================= {ticker} =================")
        results = train_ml_models_baseline(df, ticker=ticker)

        for model_name, metrics in results.items():
            final_results.append({
                "Stock": ticker,
                "Model": model_name,
                "R2": round(metrics["R2"], 4),
                "RMSE": round(metrics["RMSE"], 4),
                "MSE": round(metrics["MSE"], 6),
                "MAE": round(metrics["MAE"], 4),
            })

    # Save results
    results_df = pd.DataFrame(final_results)
    results_df.to_csv(results_path, index=False)
    print(f"💾 Saved results to {results_path}")

    return results_df

# Run training or load existing results
ml_all_results = train_all_stocks_ml_baseline(garch_data)
ml_all_results_sorted = ml_all_results.sort_values(by="R2", ascending=False)
display(ml_all_results_sorted)


================= JPM =================

📈 Training ML models for JPM...
✅ KNN — R²: -1.5234, RMSE: 0.0073, MAE: 0.0050
✅ AdaBoost — R²: 0.8564, RMSE: 0.0017, MAE: 0.0014
✅ CatBoost — R²: 0.8297, RMSE: 0.0019, MAE: 0.0014
✅ XGBoost — R²: 0.8127, RMSE: 0.0020, MAE: 0.0013
✅ RandomForest — R²: 0.9079, RMSE: 0.0014, MAE: 0.0010

================= BAC =================

📈 Training ML models for BAC...
✅ KNN — R²: -1.5992, RMSE: 0.0063, MAE: 0.0050
✅ AdaBoost — R²: 0.7429, RMSE: 0.0020, MAE: 0.0014
✅ CatBoost — R²: 0.7564, RMSE: 0.0019, MAE: 0.0014
✅ XGBoost — R²: 0.8309, RMSE: 0.0016, MAE: 0.0011
✅ RandomForest — R²: 0.6704, RMSE: 0.0023, MAE: 0.0015

================= WFC =================

📈 Training ML models for WFC...
✅ KNN — R²: -1.2453, RMSE: 0.0069, MAE: 0.0054
✅ AdaBoost — R²: 0.8091, RMSE: 0.0020, MAE: 0.0016
✅ CatBoost — R²: 0.8709, RMSE: 0.0016, MAE: 0.0012
✅ XGBoost — R²: 0.8655, RMSE: 0.0017, MAE: 0.0012
✅ RandomForest — R²: 0.8840, RMSE: 0.0016, MAE: 0.0011

===============

,Stock,Model,R2,RMSE,MSE,MAE
42,HSBC,CatBoost,0.9235,0.0014,0.000002,0.0010
44,HSBC,RandomForest,0.9168,0.0015,0.000002,0.0010
4,JPM,RandomForest,0.9079,0.0014,0.000002,0.0010
43,HSBC,XGBoost,0.9025,0.0016,0.000003,0.0011
41,HSBC,AdaBoost,0.8928,0.0017,0.000003,0.0012
14,WFC,RandomForest,0.8840,0.0016,0.000002,0.0011
37,STT,CatBoost,0.8824,0.0019,0.000004,0.0012
29,PNC,RandomForest,0.8794,0.0013,0.000002,0.0010
54,FITB,RandomForest,0.8794,0.0019,0.000004,0.0013
39,STT,RandomForest,0.8749,0.0020,0.000004,0.0013


In [8]:
from arch import arch_model
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

def evaluate_series(y_true, y_pred):
    return {
        "R2": r2_score(y_true, y_pred),
        "RMSE": mean_squared_error(y_true, y_pred, squared=False),
        "MSE": mean_squared_error(y_true, y_pred),
        "MAE": mean_absolute_error(y_true, y_pred),
    }

def forecast_volatility_arch(df, model_type="GARCH", ticker="TICKER", verbose=True):
    df = df.copy()

    # Flatten columns if MultiIndex
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = ['_'.join([str(i) for i in col if i]) for col in df.columns]

    if "Vol_target" not in df.columns or "Return" not in df.columns:
        raise KeyError(f"Missing 'Vol_target' or 'Return' in {ticker}")

    returns = df["Return"].dropna().values
    preds = []
    window_size = 500
    scale_factor = 100  # fix for scale warning

    if verbose:
        print(f"\n🔮 Running {model_type} for {ticker}...")

    for i in range(window_size, len(returns)):
        train_window = returns[i-window_size:i] * scale_factor

        try:
            if model_type == "GARCH":
                model = arch_model(train_window, vol='GARCH', p=1, q=1, dist='normal', rescale=False)
            elif model_type == "GJR":
                model = arch_model(train_window, vol='GARCH', p=1, o=1, q=1, dist='normal', rescale=False)
            elif model_type == "EGARCH":
                model = arch_model(train_window, vol='EGARCH', p=1, q=1, dist='normal', rescale=False)
            else:
                raise ValueError("Invalid model_type")

            model_fit = model.fit(disp="off")
            forecast = model_fit.forecast(horizon=1)
            pred_vol = np.sqrt(forecast.variance.values[-1][0]) / scale_factor

        except Exception as e:
            if verbose:
                print(f"⚠️ {model_type} failed at index {i}: {e}")
            pred_vol = np.nan

        preds.append(pred_vol)

        if verbose and i % 250 == 0:
            print(f"  → {model_type} | index {i} | vol: {pred_vol:.5f}")

    df[f"{model_type}_pred"] = [np.nan] * window_size + preds
    df = df.dropna(subset=["Vol_target", f"{model_type}_pred"])

    metrics = evaluate_series(df["Vol_target"], df[f"{model_type}_pred"])
    if verbose:
        print(f"✅ {model_type} for {ticker} — R²: {metrics['R2']:.4f}, RMSE: {metrics['RMSE']:.4f}, MAE: {metrics['MAE']:.4f}")

    return df, metrics

In [9]:
import os
import pandas as pd

def evaluate_all_series_models(garch_data_dict, results_path="ts_model_results.csv"):
    # If results already exist, load them
    if os.path.exists(results_path):
        print(f"📦 Loading saved time series results from {results_path}...")
        return pd.read_csv(results_path)

    results = []

    for ticker, df in garch_data_dict.items():
        for model_type in ["GARCH", "GJR", "EGARCH"]:
            print(f"\n================= {ticker} - {model_type} =================")
            try:
                _, metrics = forecast_volatility_arch(df, model_type=model_type, ticker=ticker, verbose=True)
                results.append({
                    "Stock": ticker,
                    "Model": model_type,
                    "R2": round(metrics["R2"], 4),
                    "RMSE": round(metrics["RMSE"], 4),
                    "MSE": round(metrics["MSE"], 6),
                    "MAE": round(metrics["MAE"], 4),
                })
            except Exception as e:
                print(f"⚠️ Skipping {ticker} - {model_type}: {e}")

    df_results = pd.DataFrame(results)
    df_results.to_csv(results_path, index=False)
    print(f"💾 Saved time series model results to {results_path}")

    return df_results

ts_model_results = evaluate_all_series_models(garch_data)
ts_model_results_sorted = ts_model_results.sort_values(by="R2", ascending=False)
display(ts_model_results_sorted)


================= JPM - GARCH =================

🔮 Running GARCH for JPM...
  → GARCH | index 500 | vol: 0.01690
  → GARCH | index 750 | vol: 0.01127
  → GARCH | index 1000 | vol: 0.01257
  → GARCH | index 1250 | vol: 0.01920
  → GARCH | index 1500 | vol: 0.01736
  → GARCH | index 1750 | vol: 0.01353
✅ GARCH for JPM — R²: 0.6922, RMSE: 0.0055, MAE: 0.0033

================= JPM - GJR =================

🔮 Running GJR for JPM...
  → GJR | index 500 | vol: 0.01666
  → GJR | index 750 | vol: 0.01196
  → GJR | index 1000 | vol: 0.01291
  → GJR | index 1250 | vol: 0.01614
  → GJR | index 1500 | vol: 0.01767
  → GJR | index 1750 | vol: 0.01386
✅ GJR for JPM — R²: 0.6877, RMSE: 0.0056, MAE: 0.0035

================= JPM - EGARCH =================

🔮 Running EGARCH for JPM...
  → EGARCH | index 500 | vol: 0.01579
  → EGARCH | index 750 | vol: 0.01153
  → EGARCH | index 1000 | vol: 0.01271
  → EGARCH | index 1250 | vol: 0.02082
  → EGARCH | index 1500 | vol: 0.01914
  → EGARCH | index 1750 | vo

/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


  → EGARCH | index 1750 | vol: 0.01852
✅ EGARCH for WFC — R²: 0.5503, RMSE: 0.0073, MAE: 0.0043

================= GS - GARCH =================

🔮 Running GARCH for GS...
  → GARCH | index 500 | vol: 0.01663
  → GARCH | index 750 | vol: 0.01878
  → GARCH | index 1000 | vol: 0.01377
  → GARCH | index 1250 | vol: 0.02125
  → GARCH | index 1500 | vol: 0.02081
  → GARCH | index 1750 | vol: 0.01815
✅ GARCH for GS — R²: 0.7639, RMSE: 0.0044, MAE: 0.0029

================= GS - GJR =================

🔮 Running GJR for GS...
  → GJR | index 500 | vol: 0.01574
  → GJR | index 750 | vol: 0.01514
  → GJR | index 1000 | vol: 0.01413
  → GJR | index 1250 | vol: 0.01945
  → GJR | index 1500 | vol: 0.02117
  → GJR | index 1750 | vol: 0.01818
✅ GJR for GS — R²: 0.7466, RMSE: 0.0046, MAE: 0.0030

================= GS - EGARCH =================

🔮 Running EGARCH for GS...
  → EGARCH | index 500 | vol: 0.01642
  → EGARCH | index 750 | vol: 0.01635
  → EGARCH | index 1000 | vol: 0.01484
  → EGARCH | index

/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/

  → EGARCH | index 1750 | vol: 0.01868
✅ EGARCH for GS — R²: 0.1570, RMSE: 0.0084, MAE: 0.0039

================= MS - GARCH =================

🔮 Running GARCH for MS...
  → GARCH | index 500 | vol: 0.02405
  → GARCH | index 750 | vol: 0.01561
  → GARCH | index 1000 | vol: 0.01523
  → GARCH | index 1250 | vol: 0.02452
  → GARCH | index 1500 | vol: 0.01810
  → GARCH | index 1750 | vol: 0.01808
✅ GARCH for MS — R²: 0.5941, RMSE: 0.0064, MAE: 0.0035

================= MS - GJR =================

🔮 Running GJR for MS...
  → GJR | index 500 | vol: 0.02400
  → GJR | index 750 | vol: 0.01742
  → GJR | index 1000 | vol: 0.01516
  → GJR | index 1250 | vol: 0.02316
  → GJR | index 1500 | vol: 0.01830
  → GJR | index 1750 | vol: 0.01515
✅ GJR for MS — R²: 0.5313, RMSE: 0.0068, MAE: 0.0037

================= MS - EGARCH =================

🔮 Running EGARCH for MS...
  → EGARCH | index 500 | vol: 0.02236
  → EGARCH | index 750 | vol: 0.01641
  → EGARCH | index 1000 | vol: 0.01518
  → EGARCH | index 

...

/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/p

✅ EGARCH for AXP — R²: -79523317589104196403818576147625485071367802278433842411217365154576191416517674556877602043221707705716056286715383042585088747241541933826973642842781055040415049837907516255293351150231648176763326947406416025755732950338090958233302139899506805304403534915721450661633671707145147141379394842201686016.0000, RMSE: 3486371742959041454324793795805067686438143134305763550918874898502141879987755441050061922360026488757295159676902509632595979734182158165936374284288.0000, MAE: 90654549898190915052088386537480954908024922000888963292783890544937789872304737182796420105679178602761153012516494437883609947589781552543960137728.0000

================= FITB - GARCH =================

🔮 Running GARCH for FITB...
  → GARCH | index 500 | vol: 0.01833
  → GARCH | index 750 | vol: 0.01317
  → GARCH | index 1000 | vol: 0.01539
  → GARCH | index 1250 | vol: 0.02847
  → GARCH | index 1500 | vol: 0.02161
  → GARCH | index 1750 | vol: 0.01814
✅ GARCH for FITB — R²: 0.6310, RMSE

/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


  → EGARCH | index 1750 | vol: 0.01823
✅ EGARCH for FITB — R²: 0.7327, RMSE: 0.0069, MAE: 0.0045

================= MTB - GARCH =================

🔮 Running GARCH for MTB...
  → GARCH | index 500 | vol: 0.01542
  → GARCH | index 750 | vol: 0.01387
  → GARCH | index 1000 | vol: 0.01370
  → GARCH | index 1250 | vol: 0.02472
  → GARCH | index 1500 | vol: 0.02372
  → GARCH | index 1750 | vol: 0.01995
✅ GARCH for MTB — R²: 0.6418, RMSE: 0.0067, MAE: 0.0042

================= MTB - GJR =================

🔮 Running GJR for MTB...
  → GJR | index 500 | vol: 0.01497
  → GJR | index 750 | vol: 0.01162
  → GJR | index 1000 | vol: 0.01275
  → GJR | index 1250 | vol: 0.02400
  → GJR | index 1500 | vol: 0.02398
  → GJR | index 1750 | vol: 0.01975
✅ GJR for MTB — R²: 0.6425, RMSE: 0.0067, MAE: 0.0043

================= MTB - EGARCH =================

🔮 Running EGARCH for MTB...
  → EGARCH | index 500 | vol: 0.01484
  → EGARCH | index 750 | vol: 0.01418
  → EGARCH | index 1000 | vol: 0.01389
  → EGARC

,Stock,Model,R2,RMSE,MSE,MAE
24,HSBC,GARCH,8.060000e-01,3.000000e-03,9.000000e-06,2.300000e-03
25,HSBC,GJR,8.021000e-01,3.100000e-03,9.000000e-06,2.400000e-03
9,GS,GARCH,7.639000e-01,4.400000e-03,2.000000e-05,2.900000e-03
5,BAC,EGARCH,7.473000e-01,5.200000e-03,2.700000e-05,3.500000e-03
10,GS,GJR,7.466000e-01,4.600000e-03,2.100000e-05,3.000000e-03
2,JPM,EGARCH,7.428000e-01,5.100000e-03,2.600000e-05,3.200000e-03
32,FITB,EGARCH,7.327000e-01,6.900000e-03,4.800000e-05,4.500000e-03
3,BAC,GARCH,7.226000e-01,5.400000e-03,3.000000e-05,3.600000e-03
17,PNC,EGARCH,7.194000e-01,5.500000e-03,3.000000e-05,3.600000e-03
35,MTB,EGARCH,7.108000e-01,6.000000e-03,3.600000e-05,3.900000e-03


In [14]:
def train_fusion_model(df, model_name, ts_feature="GARCH_pred", ticker="TICKER"):
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
    from sklearn.neighbors import KNeighborsRegressor
    from catboost import CatBoostRegressor
    from lightgbm import LGBMRegressor
    from xgboost import XGBRegressor

    models = {
        "KNN": KNeighborsRegressor(),
        "AdaBoost": AdaBoostRegressor(),
        "CatBoost": CatBoostRegressor(verbose=0),
        "XGBoost": XGBRegressor(verbosity=0),
        "RandomForest": RandomForestRegressor()
    }

    if model_name not in models:
        raise ValueError(f"Model '{model_name}' not recognized.")

    df = df.copy()

    # Flatten if needed
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = ['_'.join([str(i) for i in col if i]) for col in df.columns]

    if ts_feature not in df.columns:
        raise ValueError(f"'{ts_feature}' not found in DataFrame for {ticker}")

    feature_cols = [
        'RSI', 'MOM', 'OBV', 'MACD_LINE', 'MACD_SIGNAL', 'MACD_HIST',
        'STO_K', 'STO_D', 'Vol_t_1', 'Vol_t_2', 'Vol_t_3',
        'Vol_t_4', 'Vol_t_5', 'Vol_t_6', ts_feature
    ]

    X = df[feature_cols].copy()
    y = df["Vol_target"]

    split_idx = int(len(df) * 0.7)
    X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
    y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

    model = models[model_name]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    metrics = evaluate_series(y_test, y_pred)

    print(f"✅ {model_name} + {ts_feature} for {ticker} — R²: {metrics['R2']:.4f}, RMSE: {metrics['RMSE']:.4f}, MAE: {metrics['MAE']:.4f}")

    return metrics

In [15]:
import os
import pandas as pd

# Reuse existing forecast function
for ticker in tqdm(garch_data.keys()):
    for model_type in ["GJR", "EGARCH"]:
        print(f"\n📈 Adding {model_type}_pred to {ticker}...")
        df = garch_data[ticker]

        try:
            df, _ = forecast_volatility_arch(df, model_type=model_type, ticker=ticker, verbose=False)
            garch_data[ticker] = df  # Update with new column
        except Exception as e:
            print(f"⚠️ {model_type} failed for {ticker}: {e}")

def train_all_fusion_models(garch_data_dict, results_path="fusion_model_results.csv"):
    # Load existing results if file exists
    if os.path.exists(results_path):
        print(f"📦 Loading saved fusion model results from {results_path}...")
        return pd.read_csv(results_path)

    results = []

    for ticker, df in garch_data_dict.items():
        for ts_feature in ["GARCH_pred", "GJR_pred", "EGARCH_pred"]:
            if ts_feature not in df.columns:
                print(f"⚠️ Skipping {ticker} - missing {ts_feature}")
                continue

            for model_name in ["RandomForest", "XGBoost", "CatBoost", "AdaBoost", "KNN"]:
                try:
                    metrics = train_fusion_model(df, model_name, ts_feature=ts_feature, ticker=ticker)
                    results.append({
                        "Stock": ticker,
                        "Fusion_Model": f"{ts_feature}+{model_name}",
                        "R2": round(metrics["R2"], 4),
                        "RMSE": round(metrics["RMSE"], 4),
                        "MSE": round(metrics["MSE"], 6),
                        "MAE": round(metrics["MAE"], 4),
                    })
                except Exception as e:
                    print(f"⚠️ {ticker} {ts_feature}+{model_name} failed: {e}")

    fusion_df = pd.DataFrame(results)
    fusion_df.to_csv(results_path, index=False)
    print(f"💾 Saved fusion model results to {results_path}")

    return fusion_df

fusion_results_df = train_all_fusion_models(garch_data)
fusion_results_sorted = fusion_results_df.sort_values(by="R2", ascending=False)
display(fusion_results_sorted)

  0%|          | 0/12 [00:00<?, ?it/s]


📈 Adding GJR_pred to JPM...

📈 Adding EGARCH_pred to JPM...


  8%|▊         | 1/12 [00:33<06:05, 33.25s/it]


📈 Adding GJR_pred to BAC...

📈 Adding EGARCH_pred to BAC...


 17%|█▋        | 2/12 [01:07<05:40, 34.01s/it]

...

/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
 92%|█████████▏| 11/12 [06:29<00:35, 35.12s/it]


📈 Adding GJR_pred to MTB...

📈 Adding EGARCH_pred to MTB...


100%|██████████| 12/12 [07:01<00:00, 35.12s/it]


✅ RandomForest + GARCH_pred for JPM — R²: 0.8409, RMSE: 0.0019, MAE: 0.0013
✅ XGBoost + GARCH_pred for JPM — R²: 0.7580, RMSE: 0.0023, MAE: 0.0017
✅ CatBoost + GARCH_pred for JPM — R²: 0.7518, RMSE: 0.0023, MAE: 0.0020
✅ AdaBoost + GARCH_pred for JPM — R²: 0.5750, RMSE: 0.0031, MAE: 0.0026
✅ KNN + GARCH_pred for JPM — R²: -3.4382, RMSE: 0.0099, MAE: 0.0081
✅ RandomForest + GJR_pred for JPM — R²: 0.8374, RMSE: 0.0019, MAE: 0.0013
✅ XGBoost + GJR_pred for JPM — R²: 0.7710, RMSE: 0.0022, MAE: 0.0016
✅ CatBoost + GJR_pred for JPM — R²: 0.7033, RMSE: 0.0026, MAE: 0.0022
✅ AdaBoost + GJR_pred for JPM — R²: 0.6130, RMSE: 0.0029, MAE: 0.0025
✅ KNN + GJR_pred for JPM — R²: -3.4382, RMSE: 0.0099, MAE: 0.0081
✅ RandomForest + EGARCH_pred for JPM — R²: 0.8498, RMSE: 0.0018, MAE: 0.0013
✅ XGBoost + EGARCH_pred for JPM — R²: 0.7867, RMSE: 0.0022, MAE: 0.0016
✅ CatBoost + EGARCH_pred for JPM — R²: 0.7406, RMSE: 0.0024, MAE: 0.0020
✅ AdaBoost + EGARCH_pred for JPM — R²: 0.3536, RMSE: 0.0038, MAE: 0.00

,Stock,Fusion_Model,R2,RMSE,MSE,MAE
100,BMO,EGARCH_pred+RandomForest,0.9176,0.0009,0.000001,0.0007
90,BMO,GARCH_pred+RandomForest,0.9169,0.0009,0.000001,0.0007
95,BMO,GJR_pred+RandomForest,0.9141,0.0009,0.000001,0.0007
91,BMO,GARCH_pred+XGBoost,0.9073,0.0009,0.000001,0.0007
101,BMO,EGARCH_pred+XGBoost,0.9040,0.0009,0.000001,0.0007
...,...,...,...,...,...,...
170,MTB,GJR_pred+KNN,-23.2130,0.0249,0.000619,0.0211
175,MTB,EGARCH_pred+KNN,-23.2130,0.0249,0.000619,0.0211
19,BAC,GARCH_pred+KNN,-38.3875,0.0249,0.000621,0.0204
24,BAC,GJR_pred+KNN,-38.3875,0.0249,0.000621,0.0204
